## Router

We shall use the two tools we have used in `chain_messages_tool_call.ipynb` file.

Now the chat model is expected to return a normal message or tool call based on the input message.

This is a basic example of an agent, where LLM is deciding what to do.


In [1]:
from langchain_ollama import ChatOllama
llm = ChatOllama(model = "qwen2.5:7b")

In [ ]:
# Docstring is a must for a tool.....realised with an error...
def multiply(a:int, b:int)->int:
    """Can be used to multiply two numbers"""
    return a * b
def subtract(a:int, b:int)->int:
    """Can be used to subtract a number from other"""
    return a - b

llm_with_tools = llm.bind_tools([multiply, subtract])

There is a ToolNode that can be used for this purpose.

In [6]:
from langgraph.graph import START, END, StateGraph, MessagesState
from langgraph.prebuilt import ToolNode, tools_condition

def tool_calling_llm(state: MessagesState):
    return {"messages" : [llm_with_tools.invoke(state["messages"])]}

builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply, subtract]))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges("tool_calling_llm", tools_condition)
builder.add_edge("tools", END)

graph = builder.compile()

In [8]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content="2 added to 2?")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    print(m)

content='2 added to 2?' additional_kwargs={} response_metadata={} id='c3c6614d-e360-48f1-a9ef-e12ecb421e6b'
content='' additional_kwargs={} response_metadata={'model': 'qwen2.5:7b', 'created_at': '2025-04-14T09:35:13.4323994Z', 'done': True, 'done_reason': 'stop', 'total_duration': 17177273300, 'load_duration': 43662400, 'prompt_eval_count': 230, 'prompt_eval_duration': 931000000, 'eval_count': 151, 'eval_duration': 16200000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)} id='run-ea8768b1-0fc0-47ef-826f-c3d9026ddf3a-0' tool_calls=[{'name': 'subtract', 'args': {'a': 4, 'b': 2}, 'id': '18a1c876-b11e-49a0-82d2-756d676b755d', 'type': 'tool_call'}] usage_metadata={'input_tokens': 230, 'output_tokens': 151, 'total_tokens': 381}
content='2' name='subtract' id='5e5c8e7f-3bed-4a1e-9cd6-963e3d3a7078' tool_call_id='18a1c876-b11e-49a0-82d2-756d676b755d'


Well the model has successfully muffed in above input!!!